In [ ]:
import numpy as np
import pandas as pd
from qutip import basis, Qobj, sesolve
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import random
import sys
import warnings
import os
import datetime

# Suppress warnings for cleaner output (optional)
warnings.filterwarnings("ignore")

# =============================================================================
# Step 1: Read and Process the Data (Target Sequence)
# =============================================================================

def read_target_sequence(csv_file='your_data.csv', column_name='Probability_Vector'):
    """
    Reads the target sequence from a CSV file and processes it into the required format.

    Parameters:
    - csv_file (str): Path to the CSV file containing your dataset.
    - column_name (str): Name of the column containing the probability vectors.

    Returns:
    - target_sequence (list): List of numpy arrays representing the probability vectors.
    """
    print("Step 1: Reading and processing the data...")
    try:
        data_df = pd.read_csv(csv_file)
        print("Data read from CSV:")
        print(data_df.head())
    except FileNotFoundError:
        print(f"Error: '{csv_file}' not found. Please ensure the file exists in the working directory.")
        sys.exit(1)
    except pd.errors.EmptyDataError:
        print(f"Error: '{csv_file}' is empty.")
        sys.exit(1)
    except pd.errors.ParserError:
        print(f"Error: '{csv_file}' is malformed.")
        sys.exit(1)
    
    if column_name not in data_df.columns:
        print(f"Error: Column '{column_name}' not found in '{csv_file}'.")
        sys.exit(1)
    
    # Convert the probability vector column into a list of numpy arrays
    try:
        # Assuming the probability vectors are stored as strings like "[0.1, 0.2, ..., 0.0]"
        target_sequence = data_df[column_name].apply(lambda x: np.array(eval(x))).tolist()
    except Exception as e:
        print(f"Error processing probability vectors: {e}")
        sys.exit(1)
    
    print(f"Target sequence length: {len(target_sequence)}")
    return target_sequence

# =============================================================================
# Step 2: Define the Quantum System
# =============================================================================

def define_quantum_system(d=34):
    """
    Defines the quantum system's basis states.

    Parameters:
    - d (int): Dimension of the qudit (default is 34 for the 34-dimensional vectors).

    Returns:
    - basis_states (list): List of Qobj representing the basis states.
    """
    print(f"Step 2: Defining quantum system with dimension {d}...")
    basis_states = [basis(d, i) for i in range(d)]
    print(f"{d} basis states defined.")
    return basis_states

# =============================================================================
# Step 3: Define the Hamiltonian Components
# =============================================================================

def define_hamiltonian_components(basis_states, d=34, transition_coeff=1.0, perturbation_strength=0.01):
    """
    Defines projection and transition operators, and constructs the Hamiltonian.

    Parameters:
    - basis_states (list): List of Qobj representing the basis states.
    - d (int): Dimension of the qudit.
    - transition_coeff (float): Coefficient for transition operators.
    - perturbation_strength (float): Strength of random perturbations to introduce emergent behavior.

    Returns:
    - operators (list): List of projection operators for each state.
    - transition_operators (list): List of transition operators between neighboring states.
    - H_grover (function): Function defining the time-dependent Hamiltonian.
    """
    print("Step 3: Defining Hamiltonian operators for each state...")
    
    # Define projection operators for each state (0 to d-1)
    operators = [basis_states[i] * basis_states[i].dag() for i in range(d)]
    print(f"{d} projection operators defined for each state.")
    
    # Define transition operators between neighboring states with uniform coefficient
    transition_operators = []
    for i in range(d - 1):
        transition = transition_coeff * (basis_states[i] * basis_states[i + 1].dag() + basis_states[i + 1] * basis_states[i].dag())
        transition_operators.append(transition)
    
    print(f"Defined {len(transition_operators)} transition operators.")
    
    # Define the Hamiltonian function incorporating the target sequence and perturbations
    def H_grover(t, args):
        """
        Time-dependent Hamiltonian combining transition and target operators with perturbations.

        Parameters:
        - t (float): Time variable.
        - args (dict): Dictionary containing 'V' (target_sequence) and 'regressive_coeff'.

        Returns:
        - H_total (Qobj): Total Hamiltonian at time t.
        """
        V = args['V']
        regressive_coeff = args['regressive_coeff']
        t_int = int(np.floor(t))
        if t_int >= len(V):
            t_int = len(V) - 1
        # Get the probability vector at time t
        prob_vector = V[t_int]
        
        # Base Hamiltonian: sum of transition operators
        H_base = sum(transition_operators)
        
        # Weighted sum of projection operators based on the probability vector
        H_target = sum(prob_vector[i] * operators[i] for i in range(d))
        
        # Sum of regressive coefficients multiplied by their respective operators
        H_regressive = sum(regressive_coeff[i] * operators[i] for i in range(d))
        
        # Introduce a small random perturbation to promote exploration
        random_operator = Qobj(np.zeros((d, d)), dims=[[d], [d]])
        for _ in range(random.randint(0, 2)):
            i = random.randint(0, d - 1)
            j = random.randint(0, d - 1)
            if i != j and abs(i - j) > 1:
                perturbation = random.uniform(-perturbation_strength, perturbation_strength) * (
                    basis_states[i] * basis_states[j].dag() + basis_states[j] * basis_states[i].dag()
                )
                random_operator += perturbation
        
        # Total Hamiltonian: base + target + regressive + perturbation
        H_total = H_base + H_target + H_regressive + random_operator
        
        # Verify Hermiticity
        if not H_total.isherm:
            print("Warning: Hamiltonian is not Hermitian!")
        
        return H_total
    
    return operators, transition_operators, H_grover

# =============================================================================
# Step 4: Set Solver Options
# =============================================================================

def get_solver_options(atol=1e-6, rtol=1e-6, nsteps=10000, progress_bar=False):
    """
    Defines solver options as a dictionary.

    Parameters:
    - atol (float): Absolute tolerance.
    - rtol (float): Relative tolerance.
    - nsteps (int): Maximum number of integration steps.
    - progress_bar (bool): Whether to display a progress bar.

    Returns:
    - solver_options (dict): Dictionary of solver options.
    """
    solver_options = {
        'atol': atol,                # Absolute tolerance
        'rtol': rtol,                # Relative tolerance
        'nsteps': nsteps,            # Maximum number of integration steps
        'progress_bar': progress_bar # Disable progress bar for cleaner output
    }
    return solver_options

# =============================================================================
# Step 5: Initialize the Random Forest Classifier
# =============================================================================

def initialize_random_forest(n_estimators=100, max_depth=10):
    """
    Initializes the Random Forest classifier for the classical search component.

    Parameters:
    - n_estimators (int): Number of trees in the forest.
    - max_depth (int): Maximum depth of the trees.

    Returns:
    - rf_classifier (RandomForestClassifier): Initialized Random Forest classifier.
    """
    rf_classifier = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    print("Random Forest classifier initialized.")
    return rf_classifier

# =============================================================================
# Step 6: Assign Fractal Layers to Sequence Positions
# =============================================================================

def assign_fractal_layers(sequence_length, max_layers):
    """
    Assigns fractal layers to each position in the sequence.

    Parameters:
    - sequence_length (int): Length of the target sequence.
    - max_layers (int): Maximum number of fractal layers.

    Returns:
    - fractal_layers (list): List assigning each position to a fractal layer.
    """
    fractal_layers = [0] * sequence_length
    layer = 1
    step = 1
    while layer <= max_layers and step < sequence_length:
        for i in range(0, sequence_length, step * 2):
            if i + step < sequence_length:
                fractal_layers[i + step] = layer
        layer += 1
        step *= 2
    return fractal_layers

# =============================================================================
# Step 7: Evolve the Quantum-Classical Hybrid System with Fractal Layers
# =============================================================================

def evolve_system(target_sequence, basis_states, operators, transition_operators, H_grover, 
                 rf_classifier, fractal_layers, num_runs=10, max_iterations=1000, 
                 learning_rate=0.05, fractal_regressive_coeff=0.95, 
                 perturbation_strength=0.01):
    """
    Evolves the quantum-classical hybrid system using Grover’s quantum search, Random Forest, and Fractal Layers.

    Parameters:
    - target_sequence (list): List of numpy arrays representing the probability vectors.
    - basis_states (list): List of Qobj representing the basis states.
    - operators (list): List of projection operators.
    - transition_operators (list): List of transition operators.
    - H_grover (function): Function defining the time-dependent Hamiltonian.
    - rf_classifier (RandomForestClassifier): Initialized Random Forest classifier.
    - fractal_layers (list): List assigning each position to a fractal layer.
    - num_runs (int): Number of independent runs.
    - max_iterations (int): Maximum iterations per run.
    - learning_rate (float): Rate at which the regressive coefficient is adjusted.
    - fractal_regressive_coeff (float): Fixed regressive coefficient for matched positions.
    - perturbation_strength (float): Strength of random perturbations.

    Returns:
    - None
    """
    print("Step 7: Evolving the system iteratively with quantum-classical hybrid and fractal layers...")
    
    # Define solver options
    solver_options = get_solver_options()
    
    # Initialize the initial state (start with an equal superposition state)
    d = len(basis_states)
    initial_state = sum(basis_states) / np.sqrt(d)
    
    # Directory to save CSV files
    output_dir = "simulation_results"
    os.makedirs(output_dir, exist_ok=True)
    
    for run in range(1, num_runs + 1):
        print(f"\n--- Run {run} ---")
        iteration = 0
        converged = False
        
        # Initialize regressive coefficients for each state
        regressive_coeffs = {i: 1.0 for i in range(len(basis_states))}  # Start with 1.0 for all
        
        # Initialize data storage for this run
        run_data = []
        
        # Initialize a set to track matched positions
        matched_positions = set()
        
        while not converged and iteration < max_iterations:
            iteration += 1
            print(f"\nIteration {iteration}")
            
            # Update args with current regressive coefficients
            args = {
                'V': target_sequence,
                'regressive_coeff': regressive_coeffs  # Dictionary of regressive coefficients
            }

            # Define time steps for this run
            t_list = np.linspace(0, len(target_sequence) - 1, len(target_sequence))
        
            # Initialize storage for states
            states_over_time = []
        
            # Solve the system using sesolve with defined options
            try:
                result = sesolve(H_grover, initial_state, t_list, [], args=args, options=solver_options)
                states_over_time = result.states
            except Exception as e:
                print(f"An error occurred during sesolve: {e}")
                break

            # Initialize a flag to check if all positions have been assigned
            all_positions_assigned = True

            # Measure at each time step to adjust accordingly
            for idx, state in enumerate(states_over_time):
                # Calculate measurement probabilities
                measurement_probs = [abs(basis_states[i].dag() * state)[0][0][0]**2 for i in range(len(basis_states))]
                measured_state = np.argmax(measurement_probs)

                # Collect data for Random Forest
                position_data = {
                    'Run': run,
                    'Iteration': iteration,
                    'Position': idx,
                    'Target_Prob_Vector': target_sequence[idx],
                    'Measured_State': measured_state,
                    'Fractal_Layer': fractal_layers[idx],
                }
                # Add measurement probabilities as separate columns
                for state_idx, prob in enumerate(measurement_probs):
                    position_data[f'Prob_State_{state_idx}'] = prob

                run_data.append(position_data)

                # Determine if the current position is within the current fractal layer
                if fractal_layers[idx] <= iteration:
                    # Compare measured state with the highest probability in the target probability vector
                    target_state = np.argmax(target_sequence[idx])
                    if measured_state == target_state:
                        # Set regressive_coeff for matched state to fractal_regressive_coeff
                        regressive_coeffs[measured_state] = fractal_regressive_coeff
                        matched_positions.add(idx)
                        print(f"Matched state at position {idx}: {measured_state} | Set regressive_coeff to {fractal_regressive_coeff}")
                    else:
                        # If not matching, keep regressive_coeff at 1.0 to maintain influence
                        all_positions_assigned = False
                        print(f"Mismatch at position {idx}: {measured_state} != {target_state} | regressive_coeff remains at 1.0")
                else:
                    # For positions not yet assigned to the current or previous fractal layers
                    if measured_state != np.argmax(target_sequence[idx]):
                        all_positions_assigned = False

            # Print progress update
            total_positions = len(target_sequence)
            matched_count = len(matched_positions)
            print(f"\nProgress Update: {matched_count} out of {total_positions} positions have been matched.")
            
            # Check for convergence (all positions have been assigned and matched)
            if all_positions_assigned and matched_count == total_positions:
                print(f"All positions have been assigned and matched in run {run} after {iteration} iterations!")
                converged = True
                # Save the data to CSV
                df_run = pd.DataFrame(run_data)
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                csv_filename = f"Run_{run}_Data_{timestamp}.csv"
                csv_path = os.path.join(output_dir, csv_filename)
                df_run.to_csv(csv_path, index=False)
                print(f"Data for run {run} saved to '{csv_path}'.")
                break  # Exit the iteration loop

            # Train the Random Forest classifier with accumulated data
            if run_data:
                # Prepare data for training
                df_run = pd.DataFrame(run_data)
                feature_columns = [f'Prob_State_{i}' for i in range(len(basis_states))]
                X_train = df_run[feature_columns]
                y_train = df_run['Target_Prob_Vector'].apply(lambda x: np.argmax(x))
                
                rf_classifier.fit(X_train, y_train)
                print("Random Forest classifier trained with current data.")

            # Use Random Forest to predict and adjust regressive coefficients
            if run_data:
                latest_data = run_data[-1]
                latest_measurement = np.array([latest_data[f'Prob_State_{i}'] for i in range(len(basis_states))]).reshape(1, -1)
                predicted_state = rf_classifier.predict(latest_measurement)[0]
                print(f"Random Forest Prediction for next state: {predicted_state}")
                
                # Optionally, adjust regressive_coefficients based on prediction
                current_position = latest_data['Position']
                target_state = np.argmax(target_sequence[current_position])
                if predicted_state != target_state and current_position < len(target_sequence):
                    regressive_coeffs[target_state] = fractal_regressive_coeff
                    print(f"Adjusted regressive_coeff for target state {target_state} to {fractal_regressive_coeff} based on prediction.")

        # After all iterations, if not converged, save the data
        if not converged:
            print(f"Run {run} did not fully converge after {max_iterations} iterations.")
            if run_data:
                df_run = pd.DataFrame(run_data)
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                csv_filename = f"Run_{run}_Data_{timestamp}.csv"
                csv_path = os.path.join(output_dir, csv_filename)
                df_run.to_csv(csv_path, index=False)
                print(f"Partial data for run {run} saved to '{csv_path}'.")

        # After each run, plot the measurement results if converged
        if converged and run_data:
            measurements = [row['Measured_State'] for row in run_data]
            target_states = [np.argmax(seq) for seq in target_sequence]

            plt.figure(figsize=(12, 6))
            plt.plot(range(len(measurements)), measurements, label='Measured State', marker='o', markersize=4, linewidth=1)
            plt.plot(range(len(target_states)), target_states, label='Target State', linestyle='--', marker='x', markersize=4)
            plt.xlabel('Position in Sequence')
            plt.ylabel('State')
            plt.title(f'State Measurements Over Time - Run {run}')
            plt.legend()
            plt.grid(True)
            plt.tight_layout()
            plt.show()
        else:
            print(f"No plot generated for run {run}.")

# =============================================================================
# Main Execution
# =============================================================================

def main():
    # Define parameters
    csv_file = 'your_data.csv'              # CSV file containing your dataset
    column_name = 'Probability_Vector'      # Column name in the CSV file
    d = 34                                  # Dimension of the qudit (states 0-33)
    num_runs = 10                           # Number of independent runs
    max_iterations = 1000                   # Maximum iterations per run
    learning_rate = 0.05                    # Learning rate for adjusting regressive_coeff
    fractal_regressive_coeff = 0.95         # Fixed regressive coefficient upon successful match
    max_layers = 10                         # Maximum number of fractal layers
    perturbation_strength = 0.01            # Strength of random perturbations

    # Step 1: Read the target sequence
    target_sequence = read_target_sequence(csv_file, column_name)
    
    # Ensure that the probability vectors are normalized
    target_sequence = [vec / np.sum(vec) for vec in target_sequence]

    # Step 2: Define the quantum system's basis states
    basis_states = define_quantum_system(d)
    
    # Step 3: Define Hamiltonian components (projection and transition operators)
    operators, transition_operators, H_grover = define_hamiltonian_components(
        basis_states, d, transition_coeff=1.0, perturbation_strength=perturbation_strength
    )
    
    # Step 4: Initialize the Random Forest classifier
    rf_classifier = initialize_random_forest()
    
    # Step 5: Assign fractal layers to sequence positions
    fractal_layers = assign_fractal_layers(len(target_sequence), max_layers)
    print("Fractal layers assigned to sequence positions.")
    
    # Optional: Print fractal layer assignments
    # for pos, layer in enumerate(fractal_layers):
    #     print(f"Position {pos}: Layer {layer}")
    
    # Step 6: Evolve the quantum-classical hybrid system with fractal layers
    evolve_system(
        target_sequence=target_sequence,
        basis_states=basis_states,
        operators=operators,
        transition_operators=transition_operators,
        H_grover=H_grover,
        rf_classifier=rf_classifier,
        fractal_layers=fractal_layers,
        num_runs=num_runs,
        max_iterations=max_iterations,
        learning_rate=learning_rate,
        fractal_regressive_coeff=fractal_regressive_coeff,
        perturbation_strength=perturbation_strength
    )

# =============================================================================
# Entry Point
# =============================================================================

if __name__ == "__main__":
    main()
